Manga (漫画 Manga) are comics created in Japan, or by creators in the Japanese language, conforming to a style developed in Japan in the late 19th century.[1] They have a long and complex pre-history in earlier Japanese art.

Manga stories are typically printed in black-and-white,[9] although some full-color manga exist (e.g., Colorful). Colorization of Manga is usually done after it is released in black and white format. This is often avoided because colorization is time-consuming[citation needed].

To this point, the goal of this project is to aleviate this time consuming pain of colorization by using machine learning to train a model which will thereby be able to perform automatic colorization. It will train on colorized examples to identify shapes and objects in the manga image which are a consistent color. Then, when fed a black and white manga from the same artist and series, it should be able to guess the fill color by relating the properties of the shape to the training set. The same artist and series is chosen to simlify this procedure. Generalization may be part of expansion of this project if it goes well :). 

Now, what manga will we use? What a silly question! One Piece of Course! The inspiring story of a boy who eats the Gomu Gomu no mi (rubber fruit) and is going to be the Pirate King one day! If you haven't read this epic tale, not only will you get a fun glimpse of it while reading through this notebook, but I highly suggest you read the rest. 

The advantages of choosing one piece are as follows: 

1. First and most importantly, its awesome. This will make it a blast to "examine the data" so to speak. 
2. Second, there are 800 mangas chapters out so far! At an estimated 6000 polygons per page and 20 pages per chapter, this amounts to 96,000,000 million example polyons! More than enough data to train on, and we will most likely only choose a small subset of this to train on, but we can always get more. All one piece chapters are available in many different places online
3. Luffy is only 1/2 of the way through the grand line, so there is tons more manga to come. That means, that unlike complete series such as naruto which have been colorized in full already, utility may come out of the output model from this project for years to come!

Prior art. 

There have been a number of preivous attempts to do this, divided into 2 categories: 

1. Manga is often auto-colored via this technique. A section is manually colored and a "bucket fill algorithm is used
2. Machine learning auto-colorization techniques are often pixel based. They work on real life jpgs. 

This is an attempt to combine the two approaches. 

A critical juncture at this point is to note that this project processes and uses SVG images rather than the jpg format in which most manga is distributed on the web. As I began this project, I ran into the age old adage: data cleaning is the 80% of the work. Getting these polygons out of the bitmaps would require the use of openCV and a custom graph cut algorithm to seperate these polygons from their brethen. Largely because this is a pain in the ass, I began to look for other options. 1 that was staring me in the face, was the SVG format. In this format, each polygon is described as a path. If only there was an easy way to covert bitmaps into SVGs.... but wait! Vector Magic to the rescue. As most software developers know, why build a tool when you can use one. 

And so, the first step for preprocessing this data consists of batch-converting the jpg manga chapters into svg format via Vector Magic. 

Now, once this is done, we must select what features we are going to learn on. First, lets parse a sample svg and see what a standard path looks like.


In [5]:
from xml.dom import minidom
with open('sabody.svg','rb') as svg_file:
    doc = minidom.parse(svg_file)  # parseString also exists
    paths = [path.getAttribute('d') for path
                    in doc.getElementsByTagName('path')]
    doc.unlink()
    
print paths[0]   #look at the first path
    

 M 0.00 0.00 L 0.70 0.00 C 2.19 2.06 3.75 4.08 5.25 6.13 L 4.71 7.45 C 3.02 6.02 1.45 4.47 0.00 2.81 L 0.00 0.00 Z


Each path starts with an M (Move to), and consists of series of lines (L) or curves (C), finishing up with a Z, closing the path and returning it to its original starting point. Luckily the SVGs created with Vector Magic don't include any transformations or other SVG elements. What a tool!

Now, we want to format this data into a list of features. What features are we gonna pick?


To hazard the first guess, Im gonna say: 

1. Polygon length
2. Polygon area
3. Position in list
4. Position in chart
5. number of elements - complexity of the shape
6. distribution of lines vs Curves - how curvy is it?
7. Intensity

Ok so lets make a csv which has dimensionality m x (n+1), where m is the number of examples and n is the number of features, the last column will be the resulting color. We will use the CIE-LAB color space, as distances in euclidean space - are most correlated to human color perception. Shout out to https://www.lri.fr/~gcharpia/colorization_chapter.pdf who suggested that. Now, in LAB space, Since the gray level I(p) of the color C(p) at pixel p is given, we search only for the remaining 2D color, denoted by ab(p). We will run our algorithm twice, once for each of these remaining colors. 

Some of these features are not immediately available in the svg data. Thus we must turn to the third party - svg.path module of python to compute the length of each path, and indirectly, the area (using the shoelace formulat for polygons)

In [3]:
from svg.path import Path

ImportError: No module named svg.path